<div align=center>
		
<p></p>
<p></p>
<font size=5>
In the Name of God
<font/>
<p></p>
 <br/>
    <br/>
    <br/>
<font color=#FF7500>
Sharif University of Technology - Departmenet of Computer Engineering
</font>
<p></p>
<font color=blue>
Linear Algebra 
<br>
Prof. Hamid R. Rabiee
<br>
Dr. Maryam Ramezani
</font>
<br/>
<br/>
Fall 2022

</div>

<hr/>
		<div align=center>
		    <font color=red size=6>
			    <br />
Project - Q1 <br>
Watermark Images using SVD

(35% of the overall project grade)
            	<br/>
			</font>
<font size=4>
			<br/><br/>
<font color=red>
Please run all the cells.
     </font>
</font>
<br>
<font size=3 color=grey>
</font>
    </div>

Name: AmirHossein Naghi Razlighi
<br>
Student Number: 99102423

# Watermark

A watermark is an image, overlay, or text that's placed over a digital asset. Usually, the image will be monochromatic and transparent, to enable others to still view the asset. You can read about the importance of the watermark in this [link](https://www.mediavalet.com/blog/watermarks-are-important/#).

# Add Watermark

## Imports

In [1]:
import numpy as np
import cv2
# from google.colab.patches import cv2_imshow
from PIL import Image

## Load & Show Images
If you are using colab, you can upload the files by running the following cell.

In [ ]:
from google.colab import files
uploaded = files.upload()

In the following cell, load the original image and the logo from the folder add_watermark and plot them using the ```cv2``` library. 

Note: both the image and the logo should be gray-scale.

In [87]:
def close_cv_windows():
    cv2.startWindowThread()
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [88]:
original_img = cv2.imread('./add_watermark/dog.png', cv2.IMREAD_GRAYSCALE)
watermark_img = cv2.imread('./add_watermark/logo.png', cv2.IMREAD_GRAYSCALE)

##################
#   Code here    #
#   5 points     #
##################

cv2.namedWindow('Original')
cv2.namedWindow('WaterMark')

cv2.imshow('Original', original_img)
cv2.imshow('WaterMark', watermark_img)
close_cv_windows()


## Implementation [50 points]

In this part you should add watermark to the original image. You should implement the following algorithm.

1. Perform SVD on the original image (A matrix) and form the result as $USV^T$.

2. Add watermark (W matrix), muliplied by a coefficine K, to the SVs of the original matrix. (S matrix in the previous step)

3. Now, perform SVD again on matrix D.

4. Replace the matrix S in the first step, with the SVs of the previous step (the middle matrix in step 3) to get the watermarked image.



In [89]:
def svd_calculator(matrix):
    u, s, vh = np.linalg.svd(matrix)
    return u, s, vh


In [90]:
def encode_watermark(original_img, watermark_img, watermarked_coeff=0.7):
    """
    Args:
        original_img (numpy.ndarray): Original Image
        watermark_image (numpy.ndarray): Watermark Image

    Returns:
        numpy.ndarray: Watermarked Image
    """
    ##################
    #  Code here     #
    #  40 points     #
    ##################

    u, sigma, vh = svd_calculator(original_img)
    S = np.diag(sigma)

    D = watermark_img * watermarked_coeff + S 
    u_d, sigma_d, vh_d = svd_calculator(D)

    S_d = np.diag(sigma_d)

    watermarked_image = u @ S_d @ vh
    return watermarked_image


Display the watermarked image. 

In [91]:
##################
#  Code here     #
#  5 points      #
################## 

result = encode_watermark(original_img, watermark_img, watermarked_coeff=100)
cv2.imshow('Watermarked', result)
close_cv_windows()


Explain about the place you added the logo. How is the watermarked image visually different from the original image? [5 points]

Answer: 

When we add $watermarkImg * watermarkCoeff$ to S matrix, we are adding some portion of the watermark image to the singular values of original image. then we calculate singular values of the new matrix and replace the singular values of the original image with the new singular values. In this way, we can add watermark to the original image. Note that watermarked_image does not visually cotain whole logo (you can not see it!) but it's some sort of a signature! It changes the values of some singular values of the original image and so, watermarked_image is somehow different(in pixel intensities) from original_image.

# Watermark Extraction [45 points]

## Load Files

In [ ]:
from google.colab import files
uploaded = files.upload()

In [92]:
watermarked_u = np.load('./extract_watermark/watermarked_u.npy')
watermarked_vh = np.load('./extract_watermark/watermarked_vh.npy')
S = np.load('./extract_watermark/S.npy')
watermarked_img = cv2.imread(
    './extract_watermark/watermarked_image.jpg', 0)


## Implementation 

The task in this part is to extract the watermarked image from a distorted one ($A^*_w$ matrix) and write the sentence that is written in the watermarked image. The matrix of singular values (S) from the original image, $U_w$, $V^T_w$, and the watermarked image are given to you.

1. Perform SVD on the distorted watermarked image ($A^*_w$ matrix) in the form of $U^* S^*_w V^{*T}$.

2. Compute the matrix that includes the watermark by $D^* = U_w S^*_w V^T_w$.

3. Compute the corrupted watermark by subtracting the matrix $S$ from $D^*$ and dividing the result by the coefficient of the watermark, $K$.


In [93]:
def get_watermark(watermarked_img, watermarked_u, S, watermarked_vh, watermarked_coeff):
    ##################
    #  Code here     #
    #  35 points     #
    ##################
    u_star, sigma_star, vh_star = svd_calculator(watermarked_img)
    D_star = watermarked_u @ np.diag(sigma_star) @ watermarked_vh
    watermark = (D_star - S) / watermarked_coeff
    return watermark


Get the watermark logo and display it. Note that in this section we do not know the watermark coefficient. You have to run the previous cell's function for at least 5 different coefficients and compare the results.   

In [94]:
##################
#  Code here     #
#  5 points      #
##################

watermark = get_watermark(
    watermarked_img, watermarked_u, S, watermarked_vh, 100)

cv2.imshow('Distorted', watermarked_img)
cv2.imshow('Watermark', watermark)
close_cv_windows()


Explain the effect of different coefficient values on the quality of the displayed logo. [5 points]

Answer:

The quality of the logo depends on the coefficient. If the coefficient is too small, the logo will not be distinguishable. actually, there will be some white lines! If the coefficient is too big, the logo will be too dark and not distinguishable. So, we should choose the coefficient carefully. SomeHow, if we make the coefficient bigger, it looks like it's getting blurier!